In [2]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [1]:
TIME_STEPS = 30
BATCH_SIZE = 200
HIDDEN_SIZE = 100
EPOCH = 50
XFILE = 'data/imu_proc01-Feb-2018.csv'
YFILE = 'data/vicon_proc01-Feb-2018.csv'

In [2]:
def build_model(in_size, out_size):
    model = Sequential()
    model.add(LSTM(HIDDEN_SIZE, input_shape=(None, in_size)))
    model.add(Dropout(0.5))
    model.add(Dense(out_size, input_shape=(None, HIDDEN_SIZE)))
    model.add(Activation("linear"))
    model.compile(loss="mean_squared_error", optimizer="rmsprop")
    print(model.summary())
    return model

In [6]:
def get_seq_data(X, y):
    seqX = np.zeros((X.shape[0] - TIME_STEPS, TIME_STEPS, X.shape[1]))
    seqY = np.zeros((y.shape[0] - TIME_STEPS, y.shape[1]))
    for i in range(X.shape[0] - TIME_STEPS):
        seqX[i] = (X[i:i + TIME_STEPS])
        seqY[i] = y[i + TIME_STEPS]
    return seqX, seqY

In [7]:
def predict(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    print(rmse)
    print('\t'.join(map(str, r2)))

In [12]:
nrows = 4300
X = np.genfromtxt(XFILE, delimiter=',')
X = X[:nrows, 10:]  # wrist only
y = np.genfromtxt(YFILE, delimiter=',')
y = y[:nrows]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
# scaler = StandardScaler().fit(X_train)
scaler = MinMaxScaler((-1, 1)).fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train, y_train = get_seq_data(X_train, y_train)
X_test, y_test = get_seq_data(X_test, y_test)

In [13]:
X_train.shape

(3840, 30, 10)

In [43]:
model = build_model(X_train.shape[-1], y_train.shape[-1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 505       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 44,905
Trainable params: 44,905
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
model.fit(X_train, y_train, BATCH_SIZE, EPOCH, validation_split=0.05)

Train on 3648 samples, validate on 192 samples
Epoch 1/50
 200/3648 [>.............................] - ETA: 0s - loss: 28.9382

 400/3648 [==>...........................] - ETA: 0s - loss: 25.3164

 600/3648 [===>..........................] - ETA: 0s - loss: 25.4726

 800/3648 [=====>........................] - ETA: 0s - loss: 25.1853

1000/3648 [=======>......................] - ETA: 0s - loss: 24.8458

...

3000/3648 [=======================>......] - ETA: 0s - loss: 20.9762

3200/3648 [=========================>....] - ETA: 0s - loss: 21.0290

3400/3648 [==========================>...] - ETA: 0s - loss: 20.8644

3600/3648 [============================>.] - ETA: 0s - loss: 20.6617

3648/3648 [==============================] - 1s 275us/step - loss: 20.6282 - val_loss: 73.8029


In [49]:
predict(model, X_test, y_test)

93.862330534
0.2073161575	0.571466780614	0.309519055006	0.644217326833	0.13298467765


In [21]:
plt.plot(history.history['val_loss'])
plt.show()